In [1]:
import pandas as pd
import os

In [32]:
len(os.listdir('PeerRead/data/arxiv.cs.lg_2007-2017/train/parsed_pdfs'))

4543

In [39]:
all_paper=[]
for paper in os.listdir('PeerRead/data/arxiv.cs.lg_2007-2017/train/parsed_pdfs'):
    data=pd.read_json('PeerRead/data/arxiv.cs.lg_2007-2017/train/parsed_pdfs/' + paper).T.reset_index()
    data=data.iloc[1:]
    all_paper.append(data)

    # break

In [43]:
dataset=pd.concat(all_paper).reset_index(drop=True)

In [44]:
dataset.columns

Index(['index', 'source', 'title', 'authors', 'emails', 'sections',
       'references', 'referenceMentions', 'year', 'abstractText', 'creator'],
      dtype='object')

In [45]:
dataset['authors']

0                                              [Jan Motl]
1       [Nurjahan Begum, Liudmila Ulanova, Hoang Anh D...
2       [Mohammad Amin Morid, David Eccles, Olivia R. ...
3                                      [Sébastien Bubeck]
4                     [Ravid Shwartz-Ziv, Naftali Tishby]
                              ...                        
4538    [Marc Claesen, Jaak Simm, Dusan Popovic, Yves ...
4539              [Kalina Jasinska, Nikos Karampatziakis]
4540    [Tom Le Paine, Pooya Khorrami, Shiyu Chang, Ya...
4541    [Mohammad Pezeshki, Linxi Fan, Aaron Courville...
4542    [Jihun Hamm, Adam Champion, Guoxing Chen, Mikh...
Name: authors, Length: 4543, dtype: object

list